In [1]:
import xarray as xr
import intake

# Load CMIP6 catalog from Google Cloud
col = intake.open_esm_datastore(
    "https://storage.googleapis.com/cmip6/pangeo-cmip6.json"
)

# Search for one model, one scenario, one variable
query = dict(
    experiment_id=["historical", "ssp585"],
    variable_id="tas",
    table_id="Amon",
    source_id="UKESM1-0-LL",  # You can choose any model
)
cat = col.search(**query)

# Open as a single xarray dataset
ds = cat.to_dataset_dict(zarr_kwargs={"consolidated": True}, aggregate=True)



--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.table_id.grid_label'


c:\Users\ryant\AppData\Local\Programs\Python\Python312\Lib\site-packages\intake_esm\source.py:99: SerializationWarning: Unable to decode time axis into full numpy.datetime64[ns] objects, continuing using cftime.datetime objects instead, reason: dates out of range. To silence this warning use a coarser resolution 'time_unit' or specify 'use_cftime=True'.
  ds = xr.open_dataset(url, **xarray_open_kwargs)
c:\Users\ryant\AppData\Local\Programs\Python\Python312\Lib\site-packages\intake_esm\source.py:99: SerializationWarning: Unable to decode time axis into full numpy.datetime64[ns] objects, continuing using cftime.datetime objects instead, reason: dates out of range. To silence this warning use a coarser resolution 'time_unit' or specify 'use_cftime=True'.
  ds = xr.open_dataset(url, **xarray_open_kwargs)
c:\Users\ryant\AppData\Local\Programs\Python\Python312\Lib\site-packages\intake_esm\source.py:298: FutureWarning: In a future version of xarray the default value for join will change from 

In [2]:
key = list(ds.keys())[0]
data = ds[key]
data

<xarray.Dataset> Size: 657MB
Dimensions:         (member_id: 3, dcpp_init_year: 1, time: 1980, lat: 144,
                     lon: 192, bnds: 2)
Coordinates:
  * member_id       (member_id) object 24B 'r13i1p1f2' 'r14i1p1f2' 'r15i1p1f2'
  * dcpp_init_year  (dcpp_init_year) object 8B None
  * time            (time) object 16kB 1850-01-16 00:00:00 ... 2014-12-16 00:...
  * lat             (lat) float64 1kB -89.38 -88.12 -86.88 ... 86.88 88.12 89.38
  * lon             (lon) float64 2kB 0.9375 2.812 4.688 ... 355.3 357.2 359.1
    height          float64 8B 1.5
    lat_bnds        (lat, bnds) float64 2kB dask.array<chunksize=(144, 2), meta=np.ndarray>
    lon_bnds        (lon, bnds) float64 3kB dask.array<chunksize=(192, 2), meta=np.ndarray>
    time_bnds       (time, bnds) object 32kB dask.array<chunksize=(1980, 2), meta=np.ndarray>
Dimensions without coordinates: bnds
Data variables:
    tas             (member_id, dcpp_init_year, time, lat, lon) float32 657MB dask.array<chunksize=(1, 1, 889, 144, 192), meta=np.ndarray>
Attributes: (12/53)
    Conventions:                      CF-1.7 CMIP-6.2
    DODS_EXTRA.Unlimited_Dimension:   time
    activity_id:                      CMIP
    branch_method:                    standard
    branch_time_in_child:             0.0
    branch_time_in_parent:            0.0
    ...                               ...
    intake_esm_attrs:variable_id:     tas
    intake_esm_attrs:grid_label:      gn
    intake_esm_attrs:_data_format_:   zarr
    netcdf_tracking_ids:              hdl:21.14100/0fd04828-3c8a-4191-b254-0c...
    version_id:                       v20210426
    intake_esm_dataset_key:           CMIP.NIMS-KMA.UKESM1-0-LL.historical.Am...

In [3]:
# Convert to °C if in Kelvin
data["tas"] = data["tas"] - 273.15

# Select historical baseline
baseline = data["tas"].sel(time=slice("1850-01", "1900-12")).mean("time")

# Compute anomaly
anomaly = data["tas"].groupby("time.year").mean("time") - baseline

# Zonal mean (average over longitude)
zonal_mean = anomaly.mean("lon")

# Optional: smooth by latitude
zonal_mean = zonal_mean.rolling(lat=3, center=True).mean()

In [4]:
zonal_df = zonal_mean.to_dataframe().reset_index()
zonal_df

,member_id,dcpp_init_year,year,lat,height,tas
0,r13i1p1f2,NaN,1850,-89.375,1.5,NaN
1,r13i1p1f2,NaN,1850,-88.125,1.5,-0.341286
2,r13i1p1f2,NaN,1850,-86.875,1.5,-0.114779
3,r13i1p1f2,NaN,1850,-85.625,1.5,0.077651
4,r13i1p1f2,NaN,1850,-84.375,1.5,0.303654
...,...,...,...,...,...,...
71275,r15i1p1f2,NaN,2014,84.375,1.5,2.807010
71276,r15i1p1f2,NaN,2014,85.625,1.5,2.732790
71277,r15i1p1f2,NaN,2014,86.875,1.5,2.670276
71278,r15i1p1f2,NaN,2014,88.125,1.5,2.564642


In [5]:
zonal_df.to_csv("zonal_anomaly.csv", index=False)